# Libraries

In [1]:
import itk
from tqdm import tqdm
from itkwidgets import compare, checkerboard
import matplotlib.pyplot as plt
import numpy as np
import os
import SimpleITK as sitk
from tqdm import tqdm
import seaborn as sns
from pathlib import Path
from typing import List

In [ ]:
def calculate_dice_coefficient(fixed_label: itk.Image, registered_label: itk.Image) -> float:  
    # Ensure binary labels
    dice=0
    for cls in [1,2,3]:
        fixed_array = (fixed_label == cls).astype(int)
        registered_array = (registered_label == cls).astype(int)
        intersection = np.sum(fixed_array * registered_array)
        sum_fixed = np.sum(fixed_array)
        sum_registered = np.sum(registered_array)
        dice += (2.0 * intersection) / (sum_fixed + sum_registered + 1e-6) 
    return dice/3

# Registration One vs Rest param:9 affine

In [3]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.affine.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [00:58<13:41, 58.70s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.5764718672699909


Fixed Images:  13%|█▎        | 2/15 [01:42<10:47, 49.82s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.5743672019061804


Fixed Images:  20%|██        | 3/15 [02:25<09:22, 46.91s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.592401315216313


Fixed Images:  93%|█████████▎| 14/15 [10:43<00:45, 45.24s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.5522962076343844


Fixed Images: 100%|██████████| 15/15 [11:29<00:00, 45.99s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.5488733254920073

Best fixed image: 1008.nii.gz with average dice score: 0.6268816480859328


# Registration One vs Rest param:9 elastic

In [5]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.elastic.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [02:10<30:27, 130.54s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.4156139222486543


Fixed Images:  13%|█▎        | 2/15 [04:02<25:55, 119.62s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.30718936953065096


Fixed Images:  20%|██        | 3/15 [05:49<22:45, 113.83s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.33949400002379887


Fixed Images:  93%|█████████▎| 14/15 [28:59<02:03, 123.87s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.24041808190085753


Fixed Images: 100%|██████████| 15/15 [31:03<00:00, 124.21s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.2963839623782875

Best fixed image: 1013.nii.gz with average dice score: 0.4508796255660966


# Registration One vs Rest param:9 affine + elastic

In [6]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.affine.txt")
            # add elastic transformation
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.elastic.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [04:19<1:00:26, 259.06s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.7402161430061612


Fixed Images:  13%|█▎        | 2/15 [08:32<55:26, 255.88s/it]  

Average Dice Score for fixed image 1001.nii.gz: 0.7396773613127138


Fixed Images:  20%|██        | 3/15 [12:48<51:07, 255.66s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.7395430800199587


Fixed Images:  93%|█████████▎| 14/15 [1:03:30<04:42, 282.50s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.7638160672010655


Fixed Images: 100%|██████████| 15/15 [1:06:55<00:00, 267.69s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.7006415322977624

Best fixed image: 1017.nii.gz with average dice score: 0.7638160672010655


# Registration One vs Rest param:9 affine + bspline

In [9]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.affine.txt")
            parameter_object.AddParameterFile("parameters/default/Parameters_BSpline.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [05:23<1:15:35, 323.94s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.7100183597315913


Fixed Images:  13%|█▎        | 2/15 [10:28<1:07:42, 312.54s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.7097587588543124


Fixed Images:  20%|██        | 3/15 [15:38<1:02:17, 311.43s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.7135812041741622


Fixed Images:  93%|█████████▎| 14/15 [1:13:15<05:01, 301.43s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.7138086775397741


Fixed Images: 100%|██████████| 15/15 [1:18:29<00:00, 313.98s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.6797079163162565

Best fixed image: 1008.nii.gz with average dice score: 0.7221556485470869


# Registration One vs Rest param:21 affine

In [4]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/21/Par0021_affine.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [01:36<22:34, 96.72s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.5553149114608946


Fixed Images:  13%|█▎        | 2/15 [03:09<20:23, 94.15s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.5882916464795295


Fixed Images:  20%|██        | 3/15 [04:38<18:25, 92.12s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.5964670604065407


Fixed Images:  93%|█████████▎| 14/15 [22:16<01:39, 99.10s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.5668553097199521


Fixed Images: 100%|██████████| 15/15 [24:01<00:00, 96.11s/it] 

Average Dice Score for fixed image 1036.nii.gz: 0.5266673331773413

Best fixed image: 1008.nii.gz with average dice score: 0.6246598002356991


# Registration One vs Rest param:21 affine + 9 elastic

In [7]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/21/Par0021_affine.txt")
            parameter_object.AddParameterFile("parameters/9/Parameters.Par0009.elastic.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")

Fixed Images:   7%|▋         | 1/15 [05:48<1:21:20, 348.63s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.7456871766113492


Fixed Images:  13%|█▎        | 2/15 [11:19<1:13:20, 338.47s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.7396491785898343


Fixed Images:  20%|██        | 3/15 [17:05<1:08:19, 341.61s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.7433549969248041


Fixed Images:  93%|█████████▎| 14/15 [1:20:29<05:42, 342.79s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.7621701133738821


Fixed Images: 100%|██████████| 15/15 [1:24:56<00:00, 339.77s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.6959384584373465

Best fixed image: 1017.nii.gz with average dice score: 0.7621701133738821


# Registration One vs Rest param:21 affine + bspline

In [8]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/21/Par0021_affine.txt")
            parameter_object.AddParameterFile("parameters/default/Parameters_BSpline.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")


Fixed Images:   7%|▋         | 1/15 [06:12<1:26:58, 372.76s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.7064591913444962


Fixed Images:  13%|█▎        | 2/15 [12:02<1:17:47, 359.00s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.7070645635186287


Fixed Images:  20%|██        | 3/15 [17:57<1:11:28, 357.39s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.7101846517298223


Fixed Images:  93%|█████████▎| 14/15 [1:25:38<06:12, 372.56s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.7118640936920794


Fixed Images: 100%|██████████| 15/15 [1:32:33<00:00, 370.26s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.6727890331875683

Best fixed image: 1008.nii.gz with average dice score: 0.7203232045025102


# Registration One vs Rest param:default affine

In [10]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/default/Parameters_Affine.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")


Fixed Images:   7%|▋         | 1/15 [00:59<13:46, 59.01s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.5615627406443571


Fixed Images:  13%|█▎        | 2/15 [01:58<12:49, 59.17s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.5738752378753185


Fixed Images:  20%|██        | 3/15 [02:55<11:39, 58.26s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.5763721364487162


Fixed Images:  93%|█████████▎| 14/15 [13:39<00:59, 59.10s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.5557639668364839


Fixed Images: 100%|██████████| 15/15 [14:36<00:00, 58.41s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.49539124121392336

Best fixed image: 1013.nii.gz with average dice score: 0.6202864621864359


# Registration One vs Rest param:default bspline

In [11]:
best_scores = {}
folder_path = 'training-set/training-images/'
label_path = 'training-set/training-labels/'
# Iterate through each potential fixed image
for fixed_file in tqdm(os.listdir(folder_path), desc="Fixed Images"):
    if not fixed_file.endswith(('.nii', '.nii.gz')):
        continue
    fixed_image = itk.imread(os.path.join(folder_path, fixed_file), itk.F)
    fixed_label = itk.imread(os.path.join(label_path, fixed_file.replace('.nii.gz', '_3C.nii.gz')), itk.F)
    files_to_process = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.nii', '.nii.gz')) and f != fixed_file]
    with tqdm(total=len(files_to_process), desc="Registration Progress") as pbar:
        dice_scores = []
        for moving_file in files_to_process:
            moving_image = itk.imread(moving_file, itk.F)
            moving_label = itk.imread(os.path.join(label_path, os.path.basename(moving_file).replace('.nii.gz', '_3C.nii.gz')), itk.F)
            # Create a parameter object
            parameter_object = itk.ParameterObject.New()
            parameter_object.AddParameterFile("parameters/default/Parameters_BSpline.txt")
            result_image, result_transform_parameters = itk.elastix_registration_method(
                fixed_image, moving_image,
                parameter_object=parameter_object,
                log_to_console=False)
            # set the ResampleInterpolator of the result_transform_parameter to "FinalNearestNeighborInterpolator"
            result_transform_parameters.SetParameter("ResampleInterpolator", ["FinalNearestNeighborInterpolator"])
            # use the result_transform_parameters to transform the moving image
            result_label = itk.transformix_filter(moving_label, result_transform_parameters)
            # Calculate dice score between fixed label and transformed moving label
            dice_score = calculate_dice_coefficient(
                itk.GetArrayFromImage(fixed_label),
                itk.GetArrayFromImage(result_label)
            )
            dice_scores.append(dice_score)
            pbar.update(1)
    # Store average dice score for this fixed image
    avg_dice = np.mean(dice_scores)
    best_scores[fixed_file] = avg_dice
    print(f"Average Dice Score for fixed image {fixed_file}: {avg_dice}")
    
# Find the best fixed image
best_fixed_image = max(best_scores.items(), key=lambda x: x[1])
print(f"\nBest fixed image: {best_fixed_image[0]} with average dice score: {best_fixed_image[1]}")


Fixed Images:   7%|▋         | 1/15 [03:55<54:55, 235.43s/it]

Average Dice Score for fixed image 1000.nii.gz: 0.710842060002733


Fixed Images:  13%|█▎        | 2/15 [07:46<50:25, 232.72s/it]

Average Dice Score for fixed image 1001.nii.gz: 0.7095551074685268


Fixed Images:  20%|██        | 3/15 [11:37<46:27, 232.26s/it]

...

Average Dice Score for fixed image 1015.nii.gz: 0.7140419292912789


Fixed Images:  93%|█████████▎| 14/15 [1:02:14<04:11, 251.28s/it]

Average Dice Score for fixed image 1017.nii.gz: 0.7157144903146174


Fixed Images: 100%|██████████| 15/15 [1:06:34<00:00, 266.30s/it]

Average Dice Score for fixed image 1036.nii.gz: 0.6662939565952959

Best fixed image: 1008.nii.gz with average dice score: 0.7227815853322227
